In [1]:
from utils.configuration import get_config_from_json
from utils.training_utilities import set_GPU
from utils.plotting_traces import plot_traces
from seq2point.seq2point import SEQ2POINT
from refit_loader.data_loader import REFIT_Loader
import builtins
import os
import torch
from pprint import pprint

builtins.GENERAL_CONFIG = get_config_from_json(description="General Settings", config_file="configs/general_config.json")
builtins.MODEL_CONFIG = get_config_from_json(description="Model Parameters", config_file="configs/model_config.json")
builtins.TRAINING_CONFIG = get_config_from_json(description="Training Configuration", config_file="configs/training_config.json")
builtins.PLOT_CONFIG = get_config_from_json(description="Plot Settings", config_file="configs/plot_config.json")

In [2]:
refit = REFIT_Loader()

Followings are the general configuration of your experiment..
{'DATA_FOLDER': 'data/refit/', 'DATA_TYPE': '.csv', 'README_FILE': 'refit_loader/REFIT_Readme.txt', 'REFIT_HOUSES': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]}

Loading specified buildings: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21]
Parsing the readme file specified: refit_loader/REFIT_Readme.txt


In [3]:
kettle = refit.get_appliance_data(appliance="Kettle", houses=[2])

Loading data for appliance KETTLE ...
Fetching KETTLE data for House 2


In [4]:
kettle.resample(sampling_period='8s', fill_value=0.0, window_limit=1.0)

Resampling for house number:  2


In [5]:
kettle.data[2].isnull().sum()
# kettle.data[2].dropna(inplace=True)

aggregate    0
kettle       0
dtype: int64

In [6]:
import torch
import numpy as np

class DataGenerator(torch.utils.data.Dataset):
    def __init__(self, data, seq_len=10):
        
        # print(torch.Tensor(data.iloc[4]))
        # self.data = data
        self.time = data.index
        self.X = data['aggregate']
        self.y = data[data.columns[-1]]
        self.seq_len = seq_len
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        mid = int(index + (self.seq_len/2))
        # print(np.array(self.time[index:index + self.seq_len]), np.array(self.X.iloc[index:index + self.seq_len]), np.array(self.y.iloc[[mid]]))
        # return np.array(self.time[index:index + self.seq_len]), np.array(self.X.iloc[index:index + self.seq_len]), np.array(self.y.iloc[[mid]])
        print(np.array(self.X.iloc[index:index + self.seq_len]), np.array(self.y.iloc[[mid]]))
        return np.array(self.X.iloc[index:index + self.seq_len]), np.array(self.y.iloc[[mid]])


In [ ]:

def main(train_data = kettle.data[2], valid_data = kettle.data[2]):
    
    print(f"Followings are the {GENERAL_CONFIG['DESCRIPTION']} of your project..")
    pprint(GENERAL_CONFIG)
    
    ###### random seed selection in progress

    random_seed = 10



    data_generator = DataGenerator(data)

    train_dataloader = torch.utils.data.DataLoader(dataset=generator, 
                                                  batch_size=128, # how many samples per batch
                                                  num_workers=0, # how many subprocesses to use for data loading (higher = more)
                                                  shuffle=False) # shuffle the data

        
    network = SEQ2POINT().to(set_GPU())
    
    train_loss, validation_loss = network.run(train_dataloader, train_dataloader)

    plot_traces(traces = [train_loss, validation_loss], labels=['training', 'validation'], axis_labels=['Epochs', 'Loss'], title='Training Loss vs Validation Loss per Epoch')


In [ ]:
main(train_x, train_y, validate_x, validate_y)